In [ ]:
# default_exp db
# hide
_FNAME='db'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted db.ipynb.


In [ ]:
#export
import json
import time
import os
import atexit
import logging

import neo4j
import neo4j.exceptions as neo4jexceptions

from retrying import retry
logger = logging.getLogger()


DEFAULT_AUTH = 'neo4j/talosadmin'

neo_uri = os.environ.get('NEO4J_URI', 'localhost:7687')
neo_auth = os.environ.get('NEO4J_AUTH', DEFAULT_AUTH)
uri = 'bolt://{0}'.format(neo_uri)
logger.info('Connecting to {0}'.format(uri))

DEFAULT_DRIVER = None

class BoltConnectionUnsuccessful(Exception):
    pass
class MultipleSearchResults(Exception):
    pass
class NoSearchResults(Exception):
    pass

## Driver Management

In [ ]:
#export
@retry(wait_fixed=2000, stop_max_attempt_number=10)
def make_driver(uri=uri, auth=neo_auth, container=None):
    '''
    Create a driver for neo4j database.
    uri: str
        The uri to the bolt port of the database
    auth: str
        auth string in the form of user/password
    container: docker container object
        If provider, the uri will be figured out automatically by examining the container.
        uri will be ignored if container is provided
    '''
    if container:
        uri="bolt://0.0.0.0:{}".format(container.port.bolt)
    try:
        return neo4j.GraphDatabase.driver(uri, auth=tuple(auth.split('/')))
    except OSError:
        raise BoltConnectionUnsuccessful('Could not connect to the neo4j bolt server at {}'.format(uri))

def set_default_driver(driver):
    global DEFAULT_DRIVER
    DEFAULT_DRIVER = driver
        
def default_driver(uri=uri, auth=neo_auth, force_remake=False):
    global DEFAULT_DRIVER
    if force_remake:
        DEFAULT_DRIVER = None
    if DEFAULT_DRIVER is None:
        driver = make_driver(uri=uri, auth=auth)
        atexit.register(driver.close)
        set_default_driver(driver)
    else:
        driver = DEFAULT_DRIVER
    return driver

def tearDown():
    global DEFAULT_DRIVER
    if DEFAULT_DRIVER is not None:
        DEFAULT_DRIVER.close()
    DEFAULT_DRIVER = None

## Query

In [ ]:
# export

class Query(object):
    def __init__(self, driver=None):
        self.driver = default_driver() if driver is None else driver
        self.lines = []

    def add(self, line):
        self.lines.append(line)

    def _make_response(self, session, parameters=None):
        return session.run('\n'.join(self.lines), parameters=parameters)

    def results(self, parameters=None):
        '''
        This got broken as of version 4.1.  I renamed it so that it will crash
        older code on purpose.
        '''
        with self.driver.session() as session:
            resp = self._make_response(session, parameters)
            records = [r for r in resp]
        return records

    def data(self, parameters=None):
        """
        Returns all the data
        """
        try:
            with self.driver.session() as session:
                resp = self._make_response(session, parameters)
                data = resp.data()
                return data
        except TypeError as typeerror:
            raise NoSearchResults('No results found from query {q} with params {p}'.format(
                        q=self.lines, p=parameters)) from typeerror

    def single(self, parameters=None):
        """
        Returns only a single node
        """
        try:
            with self.driver.session() as session:
                resp = self._make_response(session, parameters)
                data = resp.single()[0]
                return data
        except TypeError as typeerror:
            raise NoSearchResults('No results found from query {q} with params {p}'.format(
                        q=self.lines, p=parameters)) from typeerror

    def only(self, parameters=None):
        """
        Use if you expect only a single result to come back
        """
        rd = self.data(parameters)
        if len(rd) > 1:
            raise MultipleSearchResults('Multiple entries found for query {q} with params {p}'.format(
                            q=self.lines, p=parameters))
        try:
            return rd[0]
        except IndexError as indexerror:
            raise NoSearchResults('No results found from query {q} with params {p}'.format(
                        q=self.lines, p=parameters)) from indexerror
    def create(self, parameters=None):
        """
        Use to create nodes
        Returns a summary of how many were created
        """
        with self.driver.session() as session:
            resp = self._make_response(session, parameters)
            counters = resp.consume().counters
        return counters

## Generate snippets of cypher text

In [ ]:
#export

def node(label=None, variable=None, **kwargs) -> str:
    variable = variable if variable is not None else ''
    label = label if label is not None else []
    qstr = "({var}".format(var=variable)

    if isinstance(label, str):
        label = [label]

    for lab in label:
        qstr += ":{label}".format(label=lab)

    if kwargs:
        qstr+= ' {'

    props = []
    for name, value in kwargs.items():
        props.append("{name}: {value}".format(name=name, value=json.dumps(value)))

    if kwargs:
        qstr+= ','.join(props)
        qstr+= '}'
    qstr += ')'
    return qstr

def relationship(rel_type, name=None, **kwargs) -> str:
    name = '' if name is None else name
    qstr='[{name}:{rel_type}'.format(name=name, rel_type=rel_type)
    if kwargs:
        qstr+= ' {'

    props = []
    for name, value in kwargs.items():
        props.append("{name}: '{value}'".format(name=name, value=value))

    if kwargs:
        qstr+= ','.join(props)
        qstr+= '}'
    qstr+= ']'
    return qstr

def MATCH(label, variable=None, **kwargs) -> str:
    return "MATCH {node}".format(node=node(label, variable, **kwargs))

def RETURN(*args) -> str:
    return "RETURN {0}".format(', '.join(args))

def WITH(*args) -> str:
    return "WITH {0}".format(', '.join(args))



In [ ]:
#export

def node_to_dict(resp:dict):
    return {key: dict(value) for key, value in resp.items()}

def fetch(label, **kwargs) -> dict:
    '''
    Quickly find a single node by label and properties and return it as a dictionary
    '''
    query = Query()
    var = 'var'
    query.add(MATCH(label, var, **kwargs))
    query.add(RETURN(var))
    return node_to_dict(query.only())[var]

def set_properties(label:str, search_properties:dict, **kwargs):
    '''
    Find a node of and set properties to it
    label: string of the node Label
    search_properties: dict of properties the existing node must have.  e.g. eid="abc123"
    additional kwargs will be applied to the node
    '''
    query = Query()
    var = 'var'
    query.add(MATCH(label, var, **search_properties))
    for prop, val in kwargs.items():
        query.add('SET var.{prop}={jsonval}'.format(
                    prop=prop,
                    jsonval=json.dumps(val)))
    resp = query.create()
    return resp

## Common actions

In [ ]:
#export

def unlink(source: str, target:str, relationship:str):
    '''
    Delete the relationship between the source and target nodes
    (source)-[:RELATIONSHIP]->(target)
    source: str
        Cypher string of source node
    target: str
        Cypher string of target node
    relationship: str
        Type of relationship to delete
        
    Returns a query response
    '''
    query = Query()
    query.add("MATCH {source}-[rel:{rel}]->{target}".format(source=source, rel=relationship, target=target))
    query.add("DELETE rel")
    query.add(RETURN('rel'))
    return query.create()

def get_properties_of(label, properties, **kwargs):
    '''
    label: str
        The label of the node youre looking for.  e.g. Equipment
    properties: iterable
        A list or tuple of the strings of property names
    Any additional kwargs will go to searching for the node.  e.g. {"eid": "device_eid"}
    Returns list of the properties of the node.  Node must be unique.
    '''
    query = Query()
    query.add(MATCH(label, 'var', **kwargs))
    returnprops = ['var.{}'.format(prop) for prop in properties]
    query.add(RETURN(','.join(returnprops)))

    try:
        resp = query.only()
    except NoSearchResults:
        return []
    return [resp['var.{}'.format(prop)] for prop in properties]

def link(node1, node1name, node2, node2name, rel_type, **rel_kwargs):
    """
    Create relationship (node1)-[:rel_type]->(node2)
    node1: str describing a cypher node
    node1name: the variable name used in node1
    rel_type: the type of relationship we are creating from node1 to node2
    rel_kwargs: the properties you want the relationship to have, if any
    """
    query = Query()
    query.add("MATCH {}".format(node1))
    query.add("WITH {}".format(node1name))
    query.add("MATCH {}".format(node2))
    rel_str = relationship(rel_type, name=None, **rel_kwargs)
    query.add("MERGE ({n1n})-{rel_str}->({n2n})".format(n1n=node1name, rel_str=rel_str, n2n=node2name))
    resp = query.create()
    return resp